In [1]:
from dunkerquesprint import Portic
from ipyleaflet import Map, Marker, Popup, basemaps, basemap_to_tiles, CircleMarker
from ipywidgets import HTML

client = Portic()

# this function allows to map a value from a domain of min-max to another
def map_value(value, domain_min, domain_max, range_min, range_max):
    left_span = domain_max - domain_min
    right_span = range_max - range_min

    # Convert the left range into a 0-1 range (float)
    scaled = float(value - domain_min) / float(left_span)

    # Convert the 0-1 range into a value in the right range.
    return range_min + (scaled * right_span)

# doc : https://ipyleaflet.readthedocs.io/en/latest/api_reference/circle_marker.html

# Flows

In [3]:
flows = client.get_flows(year=1789)
print(flows[0:1])

[{'travel_id': '0000008N- 55', 'distance_dep_dest': 600.662, 'distance_homeport_dep': 199.814, 'departure': 'Dunkerque', 'departure_fr': 'Dunkerque', 'departure_en': 'Dunkirk', 'departure_uhgs_id': 'A0204180', 'departure_latitude': '51.05', 'departure_longitude': '2.366667', 'departure_admiralty': 'Dunkerque', 'departure_province': 'Flandre', 'departure_states': '[{"1749-1815" : "France"}]', 'departure_substates': None, 'departure_state_1789_fr': 'France', 'departure_substate_1789_fr': None, 'departure_state_1789_en': 'France', 'departure_substate_1789_en': None, 'departure_ferme_direction': None, 'departure_ferme_direction_uncertainty': None, 'departure_ferme_bureau': None, 'departure_ferme_bureau_uncertainty': None, 'departure_partner_balance_1789': 'Dunkerque', 'departure_partner_balance_supp_1789': 'France', 'departure_partner_balance_1789_uncertainty': 0.0, 'departure_partner_balance_supp_1789_uncertainty': 0, 'departure_shiparea': 'NOR-DOVE', 'departure_status': 'siège amirauté',

Champs de *flows* avec coordonnées géo :

- `homeport_latitude`
- `homeport_longitude`
- `departure_latitude`
- `departure_longitude`
- `destination_latitude`
- `destination_longitude`

Ex :

```python
marker = Marker(
    location=(flow["homeport_latitude"], flow["homeport_longitude"])
)
```

In [21]:
center = (47, -1) # France
portic_map = Map(center=center,zoom=5)

for flow in [flows[1], flows[16056], flows[20034], flows[30032]]:
    marker = Marker(location=(flow["departure_latitude"], flow["departure_longitude"]))
    popup = HTML()
    popup.value = flow["departure"]
    popup = Popup(
        location=(flow["departure_latitude"], flow["departure_longitude"]),
        child=popup
    )
    marker.popup = popup

    portic_map.add_layer(marker)
          
display(portic_map)

Map(center=[47, -1], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

In [5]:
ports = {}

for flow in flows:
    if flow["departure_uhgs_id"] in ports:
        ports[flow["departure_uhgs_id"]]["count"] += 1
    else:
        ports[flow["departure_uhgs_id"]] = {
             "count": 1,
             'departure_fr': flow['departure_fr'],
             'departure_en': flow['departure_en'],
             'latitude': flow['departure_latitude'],
             'longitude': flow['departure_longitude'],
             "id": flow["departure_uhgs_id"]
        }

ports = [port for key,port in ports.items()]

watercolor = basemap_to_tiles(basemaps.Stamen.Watercolor)

center = (45.6876849, -1.15)

portic_map = Map(layers=(watercolor, ), center=center, zoom=5)

max_count = max([port["count"] for port in ports])
min_count = min([port["count"] for port in ports])
MIN_RADIUS = 1
MAX_RADIUS = 20

for port in ports:
    circle_marker = CircleMarker()
    circle_marker.location = (port["latitude"], port["longitude"])
    circle_marker.radius = int(map_value(port["count"], min_count, max_count, MIN_RADIUS, MAX_RADIUS))
    circle_marker.color = "red"
    circle_marker.fill_color = "red"
    portic_map.add_layer(circle_marker)

portic_map

Map(center=[45.6876849, -1.15], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

# Pointcalls

In [22]:
pointcalls = client.get_pointcalls(year=1789)
print(pointcalls[0:1])

[{'pkid': 70354, 'record_id': '00282679', 'pointcall': 'Caen', 'pointcall_uhgs_id': 'A0132409', 'toponyme_fr': 'Caen\xa0', 'toponyme_en': 'Caen\xa0', 'latitude': '49.183333', 'longitude': '-0.35', 'pointcall_admiralty': 'Caen', 'pointcall_province': 'Normandie', 'pointcall_states': '[{"1749-1815" : "France"}]', 'pointcall_substates': None, 'pointcall_states_en': '[{"1749-1815" : "France"}]', 'pointcall_substates_en': None, 'state_1789_fr': 'France', 'state_1789_en': 'France', 'substate_1789_fr': None, 'substate_1789_en': None, 'nb_conges_1787_inputdone': None, 'nb_conges_1787_cr': None, 'nb_conges_1789_inputdone': None, 'nb_conges_1789_cr': None, 'pointcall_status_uncertainity': 0.0, 'pointcall_status': 'siège amirauté', 'shiparea': 'MAN-WIGH', 'pointcall_point': '0101000020110F000022A2004C3A06E3C0EC5C5BA73A0E5841', 'ferme_direction': 'Caen', 'ferme_direction_uncertainty': 0.0, 'ferme_bureau': 'Caen', 'ferme_bureau_uncertainty': 0.0, 'partner_balance_1789': None, 'partner_balance_supp_

Champs de *pointcalls* avec coordonnées géo :

- `latitude`
- `longitude`
- `homeport_latitude`
- `homeport_longitude`